# Simulation (ctobssim) 
## (Output saved on disk)

The purpose of this notebook is to perform a simulation using `ctobbsim` (and other scripts), create a count map with `ctskymap`, fit the spectrum with `cssens` and plot the spectrum using a function in the script `make_spectrum` (which comes together with ctools). 

In order to do the simulation, one has two options:
- create an *observation container* file with the [given notebook](observation_definition.ipynb), in which many different pointings can be specified and might be useful for some specific analysis;
- perform a simpler simulation, setting everything directly in this notebook.

In [ ]:
import gammalib
import ctools
import cscripts 

# 1) Model Creation


We need to provide a **model** for the source that we want to simulate; since many sources are simulated together, the xml model will have a section for every source to simulate, including the background. An example model is in $CTOOLS/share/models/crab.xml.

In [ ]:
#!xmllint $CTOOLS/share/models/crab.xml

The simplest way to create the model of the source is by hand, editing an existing model (for example the Crab model) and adjusting the parameters to the desired value. This approach is useful for a first test, but then one need to have a way of creating XML models rapidly and in a simple way.

In order to do so, we have written a **Python** script which read in input all the data concerning the source that one wants to simulate (both the source itself and the background) and produces the XML model to be given to ctools. 

The script can bu used both with the notebook, that you find in sexten_2017/event_creator/Model_Creation.ipynb, or by running the python script from command line ($ python scriptModel_variable.py fileinput.txt). Please read the README that you find in the `model_creator` folder (you can find it on [this link](https://github.com/sharingan90/sexten_2017/blob/master/model_creator/README.md)).


# 2) Set up the simulation

The object sim in created and destroyed once the method execute() is called. If the observation definition file is provided, ctobssim will save a fits file for every pointing and a global *observation definition XML file*, different from the output of csobsdef (out_csobsdef.xml), which has a observation item for each pointing, in which the name of the fits file of the events is specified. 

## NOTE: choose just one of the following options.
#### The simulation can be set up in two main different ways: 

## - **WITH** *the observation definition XML* file (built with  [this](observation_definition.ipynb) notebook)...

### a) using the gammalib library

In [ ]:
obs_cont_bool = True
src_model = "crab.xml"
caldb, irf = 'prod2','South_0.5h'

obs = gammalib.GObservations('out_csobsdef.xml')
obs.models(gammalib.GModels("${CTOOLS}/share/models/crab.xml"))
sim = ctools.ctobssim(obs)

src_name = obs[0].name()

#sim["prefix"] = src_name+"_obs_out_"                    # this is optional: "sim_events_" is the default

sim_out = src_name+"_obs_out.xml"
sim["outevents"] = sim_out
sim.execute()

### b) OR using only ctools

In [ ]:
obs_cont_bool = True
src_model = "crab.xml"

models  = gammalib.GModels(gammalib.expand_env(src_model))
src_name = models[0].name()
print(src_name)

caldb, irf = 'prod2','South_0.5h'

sim = ctools.ctobssim()
sim["inobs"] = "out_csobsdef.xml"
sim["inmodel"] = src_model   #we load a sample model
sim_out = src_name+"_obs_out.xml"

sim["outevents"] = sim_out
sim.execute()

### ...and both in case a) and b) one can also see the resulting xml file, together with the fits files, one for each pointing. 

In [ ]:
!ls  *_obs_out.xml sim_events_*.fits

## - WITHOUT the observation definition XML file,

(here we don'use csobsutils since it can only save the results in memory and not in disk, see the notebook realted to the simulation in memory for further details)

### c)  using directly ctobssim.

In [ ]:
dur = 1800                         # seconds
emin, emax = 0.05, 20.0          # in TeV
rad, deadc = 8.0, 0.95
caldb, irf = 'prod2','South_0.5h'

# crab model
src_model = "crab.xml"

models  = gammalib.GModels(gammalib.expand_env(src_model))
src_name = models[0].name()
ra      = models[0]["RA"].value()
dec     = models[0]["DEC"].value()
print(src_name, ra, dec)

# name of the output of the simulation
sim_out = src_name+"_out_sim.fits"

In [ ]:
obs_cont_bool = False

sim = ctools.ctobssim()
sim["outevents"] = sim_out
sim["inmodel"] = src_model
sim["ra"] = ra
sim["dec"] = dec
sim["rad"] = rad
sim["tmin"] = 0.0
sim["tmax"] = dur
sim["emin"] = emin
sim["emax"] = emax
sim["caldb"] = caldb
sim["irf"] = irf
sim.execute()

### And the result is saved as fits file.

# 3) View the results
## 3.1) Create a count map with the tool ctskymap.

In [ ]:
name_map_out = "skymap.fits"

skymap = ctools.ctskymap()
skymap['inobs'] = sim_out
skymap['emin'] = 0.1
skymap['emax'] = 100
skymap['nxpix'] = 20
skymap['nypix'] = 20
skymap['binsz'] = 0.7
skymap['coordsys'] = "CEL"
skymap['proj'] = "AIT"
skymap['xref'] = 83.63 #RA
skymap['yref'] = 22.01 #DEC
skymap['bkgsubtract'] = "NONE" 
skymap['outmap'] = name_map_out
skymap.execute()

and we view it with ds9:

In [ ]:
!ds9 -wcs icrs -wcs skyformat degrees skymap.fits

Another option is to view it using matplotlib and the astropy library

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from astropy.io import fits

hdu_list = fits.open(name_map_out)
image_data = hdu_list[0].data
plt.imshow(image_data, cmap='gray')#, interpolation = 'sinc')#, vmin=0)
plt.colorbar()
#plt.savefig("out.png")

# 4) Likelihood

Now we want to compute the likelihood of the source. We will perform this calculation with [ctlike](http://cta.irap.omp.eu/ctools/users/reference_manual/ctlike.html): this tool will fit all the parameters in the xml model that have **free = "1"**. 

In the output xml model we will have both the **value** of the fitted parameters and the associated **error**, and also the Test Statistics value (if specified in the xml model (see below)). 

In [ ]:
'''
Change by hand. From:
<source name="Crab" type="PointSource"> 
to:
<source name="Crab" type="PointSource" tscalc = "1">
'''

!cp crab.xml crab_in_like.xml 
!gedit crab_in_like.xml

In [ ]:
# check whether the ts calculation has been included in the model

new_models = gammalib.GModels('crab_in_like.xml')
print((new_models[0].tscalc()))

In [ ]:
like = ctools.ctlike()
like['inobs'] = sim_out
like['inmodel'] = 'crab_in_like.xml'
like['outmodel'] = src_name+"_like_out.xml"
like.execute()

In [ ]:
#!xmllint Crab_like_out.xml 
new_models = gammalib.GModels(src_name+"_like_out.xml")

print("Model %s has TS: %.3f" % (src_name, new_models[0].ts()))

# 5) Butterfly Plot
#### The  input source model is the output from ctlike. If this is not the case, the option `fit` must be set equal `yes`.

In [ ]:
btfy = ctools.ctbutterfly()
btfy['inobs'] = "Crab_out_sim.fits"
btfy['inmodel'] = "Crab_like_out.xml"
btfy['srcname'] = src_name
btfy['caldb'] = caldb
btfy['irf'] = irf
btfy['outfile'] = 'butterfly_crab.txt'
btfy['emin'] = emin
btfy['emax'] = emax
btfy.execute()

In [ ]:
!ls -ltr butterfly_crab.txt

Since the script does not display the images in terminal, we have to open a terminal window, source again `gammalib_ctools.sh` and run `$CTOOLS/share/examples/python/show_butterfly.py simulation/butterfly_crab.txt`